# HW4
# Qianle Tang

# Problem 1

### a)

$$dC_t = \frac{\partial C}{\partial S_t}dS + \frac{\partial C}{\partial t}dt + \frac{1}{2}\frac{\partial^2 C}{(\partial S_t)^2}(dS_t)^2 $$
 the drift term will be:
 
$$\frac{\partial C}{\partial t} + \frac{1}{2}\sigma^2S_t^{2(1+\alpha)}\frac{\partial^2 C}{(\partial S_t)^2}$$

set it equal to $$\frac{\partial C}{\partial t} + \frac{1}{2}\sigma^2S_t^{2(1+\alpha)}\frac{\partial^2 C}{(\partial S_t)^2} = rC$$

with:
$$C(S_T , T ) = (K - S_T )^+$$

### b)

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

In [2]:
class CEV:
    
    def __init__(self,volcoeff,alpha,rGrow,r,S0):
        self.volcoeff = volcoeff
        self.alpha = alpha
        self.rGrow = rGrow  
        self.r = r
        self.S0 = S0


In [3]:
hw4dynamics = CEV(volcoeff=3, alpha=-0.5, rGrow=0, r=0.05, S0=100)

In [4]:
class Put:
    
    def __init__(self,T,K):
        self.T = T;
        self.K = K;

In [5]:
hw4contract=Put(T=0.25,K=100)

In [26]:
class FD_CrankNicolson:
    
    def __init__(self,SMax,SMin,deltaS,deltat):
        self.SMax=SMax
        self.SMin=SMin
        self.deltaS=deltaS
        self.deltat=deltat        
        
        
    #You complete the coding of this function:
        
    def price_put_CEV(self,contract,dynamics):
    # returns array of all initial spots,
    # and the corresponding array of put prices

        alpha, r, rGrow, volcoeff = dynamics.alpha, dynamics.r, dynamics.rGrow, dynamics.volcoeff 

        # SMin and SMax denote the smallest and largest S in the _interior_.
        # The boundary conditions are imposed one level _beyond_, 
        # e.g. at S_lowboundary=SMin-deltaS, not at SMin.
        # To relate to lecture notation, S_lowboundary is S_{-J}
        # whereas SMin is S_{-J+1}

        N=round(contract.T/self.deltat)
        if abs(N-contract.T/self.deltat)>1e-12:
            raise ValueError('Bad time step')
        numS=round((self.SMax-self.SMin)/self.deltaS)+1
        if abs(numS-(self.SMax-self.SMin)/self.deltaS-1)>1e-12:
            raise ValueError('Bad time step')
        S=np.linspace(self.SMax,self.SMin,numS)    #The FIRST indices in this array are for HIGH levels of S
        S_lowboundary=self.SMin-self.deltaS

        putprice=np.maximum(contract.K-S,0)

        ratio1 = self.deltat/self.deltaS
        ratio2 = self.deltat/self.deltaS**2
        f =    0.5*(volcoeff**2)*(S**(2+2*alpha))
        g =    rGrow*S 
        h =    -r
        F = 0.5*ratio2*f + 0.25*ratio1*g
        G =     ratio2*f - 0.50*self.deltat*h
        H = 0.5*ratio2*f - 0.25*ratio1*g

        #Right-hand-side matrix
        RHSmatrix = diags([H[:-1], 1-G, F[1:]], [1,0,-1], shape=(numS,numS), format="csr")

        #Left-hand-side matrix
        LHSmatrix = diags([-H[:-1], 1+G, -F[1:]], [1,0,-1], shape=(numS,numS), format="csr")
        # diags creates SPARSE matrices

        for t in np.arange(N-1,-1,-1)*self.deltat:

            rhs = RHSmatrix * putprice

            #Now let's add the boundary condition vectors.
            #They are nonzero only in the last component:
            rhs[-1]=rhs[-1]+2*H[-1]*(contract.K-S_lowboundary)

            putprice =  spsolve(LHSmatrix, rhs)#You code this.  Hint...
            # numpy.linalg.solve, which expects arrays as inputs,
            # is fine for small matrix equations, and for matrix equations without special structure.
            # But for large matrix equations in which the matrix has special structure,
            # we may want a more intelligent solver that can run faster 
            # by taking advantage of the special structure of the matrix.
            # Specifically, in this case, let's try to use a solver that recognizes the SPARSE MATRIX structure.
            # Try spsolve, imported from scipy.sparse.linalg
            
            putprice = np.maximum(putprice, contract.K-S)

        return(S, putprice)

In [7]:
hw4FD = FD_CrankNicolson(SMax=200,SMin=50,deltaS=0.1,deltat=0.0005)

In [8]:
(S0_all, putprice) = hw4FD.price_put_CEV(hw4contract,hw4dynamics)

In [9]:
# pricer_put_CEV_CrankNicolson gives us option prices for ALL S0 from SMin to SMax
# But let's display only for a few S0 near 100:

displayStart = hw4dynamics.S0-hw4FD.deltaS*1.5 
displayEnd   = hw4dynamics.S0+hw4FD.deltaS*1.5
displayrows  = (S0_all>displayStart) & (S0_all<displayEnd)
np.set_printoptions(precision=4, suppress=True)
print(np.stack((S0_all, putprice),axis=1)[displayrows])

[[100.1      5.8704]
 [100.       5.9183]
 [ 99.9      5.9665]]


### c)

$$\Delta = \frac{C_0^{S_0=100.1}-C_0^{S_0=99.9}}{2\Delta S} = (5.8704-5.9665)/0.2 = -0.4805$$

$$\Gamma = \frac{C_0^{S_0=100.1}-2C_0^{S_0=100}+C_0^{S_0=99.9}}{\Delta S^2} = 0.03$$

### d)

In [27]:
GBMdynamics = CEV(volcoeff=0.3, alpha=0, rGrow=0.05, r=0.05, S0=100)

In [28]:
(S0_all_GBM, putprice_GBM) = hw4FD.price_put_CEV(hw4contract,GBMdynamics)

In [29]:
displayStart = GMBdynamics.S0-hw4FD.deltaS*1.5 
displayEnd   = GMBdynamics.S0+hw4FD.deltaS*1.5
displayrows  = (S0_all_GBM>displayStart) & (S0_all_GBM<displayEnd)
#np.set_printoptions(precision=4, suppress=True)
print(np.stack((S0_all_GBM, putprice_GBM),axis=1)[displayrows])

[[100.1      5.3973]
 [100.       5.442 ]
 [ 99.9      5.487 ]]


# Problem 2

### a)

$$\Delta = N(d1)$$
so:
$$d1 = inv\Phi(\Delta)$$


where d1 has a specific form in the function:

In [32]:
def BScallPrice(sigma,S,r,K,T): 
    sd = sigma*np.sqrt(T)
    F = S*np.exp(r*T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return S*norm.cdf(d1)-K*norm.cdf(d2)*np.exp(-r*T)

so $$K = \frac{S_0e^{rT}}{e^{\sigma T^{0.5} N^{-1}(\Delta)-0.5\sigma^2T}}$$

### b)

In [33]:
import numpy as np
from scipy.stats import norm

In [34]:
def BScallPrice(sigma,S,r,K,T): 
    sd = sigma*np.sqrt(T)
    F = S*np.exp(r*T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return S*norm.cdf(d1)-K*norm.cdf(d2)*np.exp(-r*T)

In [35]:
def KfromDelta(S,sigma,r,T,delta):
    invN = norm.ppf(delta)
    num = S*np.exp(r*T)
    den = np.exp(sigma*np.sqrt(T)*invN - 0.5*sigma**2*T) 
    return num/den

In [36]:
K1 = KfromDelta(300,0.4,0.01,1/12,0.25)
K2 = KfromDelta(300,0.4,0.01,1/12,0.75)
C1 = BScallPrice(0.4,300,0.01,K1,1/12)
C2 = BScallPrice(0.4,300,0.01,K2,1/12)

In [37]:
#delta = 0.25
[K1, C1]

[326.7403577236786, 4.882592053953928]

In [38]:
#delta = 0.75
[K2, C2]

[279.61093160299833, 26.103562887425056]

### c)

In [39]:
# for 25-delta call:
0.25*300/4.882592053953928

15.36069349460906

In [40]:
# for the 75-delta call:
0.75*300/26.103562887425056

8.619513013236592

25-delta call gives higher leverage